# Data Fingerprinting in Etiq

Fingerprinting is a way to show how two datasets relate.

## Use Cases:

Oftentimes data scientists or analysts pick up issues with data based on higher level aggregates, e.g. claimed amounts for april for a client is higher than it should be or lower, etc.
We can add these types of aggregates/pivot tests incorporated in our testing suite with some options to filter on different features.

* Testing whether a dataset matches the characteristics (fingerprint) of a new one.
* Determining whether a transformed dataset has the correct number of rows based on the original one.
* "Fingerprinting" a dataset to see what characteristics it has.
* Test these expectations and create issues if any fail.


## Metrics:

The following metrics are determined for each column in both datasets, though this list can be limited if required.

These metrics will only be applied to features of a suitable type.

| Metric Name | Description | Per Table or Per Feature? |
| --- | --- | --- |
| count | How many rows are there in the dataset? | Table |
| min | Minimum value | Feature |
| max | Maximum value | Feature |
| mean | Mean value | Feature | 
| median | Median value | Feature |
| missing | How many rows are missing values? | Feature |
| sum | Sum of values | Feature |
| unique | How many unique values? | Feature |
| std | Standard Deviation | Feature |

**Table 1:** Metric names and descriptions.


## Getting Started

Let's load our datasets. For this example we have some synthetic data - insurance claims, insurance premiums and a profitability dataset which has been derived from the other two:

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

datapath = Path("./Data")
claims_df = pd.read_csv(datapath / "claims.csv")
premiums_df = pd.read_csv(datapath / "premiums.csv")
profitability_df = pd.read_csv(datapath / "profitability.csv")

In [2]:
# Claims made per client
claims_df

,ClaimID,ClientID,Month,Amount
0,A001,C01,1,1.0
1,A002,C02,2,2.0
2,A003,C01,1,2.0
3,A004,C03,3,2.0
4,A005,C05,4,0.5
5,A006,C05,6,0.5
6,A007,C01,11,0.5
7,A008,C02,12,0.5
8,A009,C02,8,0.5
9,A005,C04,9,0.5


In [3]:
# Premiums paid per customer
premiums_df.head(10)

,ClientID,Month,PremiumPaid
0,C01,1,0.1
1,C01,2,0.1
2,C01,3,0.1
3,C01,4,0.1
4,C01,5,0.1
5,C01,6,0.1
6,C01,7,0.1
7,C01,8,0.1
8,C01,9,0.1
9,C01,10,0.1


In [4]:
# Profit per customer - total premiums minus any claims.
profitability_df

,ClientID,Amount,PremiumPaid
0,C01,3.5,1.2
1,C02,3.0,1.2
2,C03,2.0,1.2
3,C04,0.5,1.2
4,C05,1.0,1.2


For each dataset, we wrap them in our Etiq adapter:

In [5]:
import etiq

claims_data = etiq.SimpleDatasetBuilder.datasets(validation_features=claims_df)
profitability_data = etiq.SimpleDatasetBuilder.datasets(validation_features=profitability_df)
premiums_data = etiq.SimpleDatasetBuilder.datasets(validation_features=premiums_df)


Thanks for using the ETIQ.AI toolkit

Help improve our product: Call `etiq.enable_telemetry()` to provide
anonymous library usage statistics.
        
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])

See 

Create our project:

In [6]:
project = etiq.projects.open(name="Fingerprint Project")

Create a snapshot for each dataset. In etiq, a snapshot will have many methods for testing data issues:

In [7]:
profitability_snapshot = project.snapshots.create(name="Profitability", dataset=profitability_data, model=None)
claims_snapshot = project.snapshots.create(name="Claims", dataset=claims_data, model=None)
premiums_snapshot = project.snapshots.create(name="Premiums", dataset=premiums_data, model=None)

INFO:etiq.charting:Histogram summary already created for this data.
INFO:etiq.charting:Histogram summary already created for this data.
INFO:etiq.charting:Histogram summary already created for this data.


Now etiq knows about our datasets, we can start to compare. How does our profitability data compare to our premiums data?

## Pivot

In [8]:
segments, issues, aggregate_issues = profitability_snapshot.scan_fingerprints(premiums_snapshot)

print("## Issues")
display(issues)
print("## Aggregate Issues")
display(aggregate_issues)

INFO:etiq.pipeline.BasePipeline0623:Starting pipeline
INFO:etiq.pipeline.BasePipeline0623:Completed pipeline
## Issues


,name,feature,segment,measure,measure_value,metric,metric_value,threshold,value,record
0,pivot,PremiumPaid,all,None,NaN,count,6.000000e+01,"(0.99, 0.99)",None,None
1,pivot,PremiumPaid,all,None,NaN,min,1.000000e-01,"(0.99, 0.99)",None,None
2,pivot,PremiumPaid,all,None,NaN,max,1.000000e-01,"(0.99, 0.99)",None,None
3,pivot,PremiumPaid,all,None,NaN,mean,1.000000e-01,"(0.99, 0.99)",None,None
4,pivot,PremiumPaid,all,None,NaN,median,1.000000e-01,"(0.99, 0.99)",None,None
5,pivot,PremiumPaid,all,None,NaN,std,4.198471e-17,"(0.99, 0.99)",None,None
6,pivot,ClientID,all,None,NaN,count,6.000000e+01,"(0.99, 0.99)",None,None


## Aggregate Issues


,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,pivot,count,None,{PremiumPaid},{all},1,1,"(0.99, 0.99)"
1,pivot,min,None,{PremiumPaid},{all},1,1,"(0.99, 0.99)"
2,pivot,max,None,{PremiumPaid},{all},1,1,"(0.99, 0.99)"
3,pivot,mean,None,{PremiumPaid},{all},1,1,"(0.99, 0.99)"
4,pivot,median,None,{PremiumPaid},{all},1,1,"(0.99, 0.99)"
5,pivot,missing,None,{PremiumPaid},{},1,0,"(0.99, 0.99)"
6,pivot,sum,None,{PremiumPaid},{},1,0,"(0.99, 0.99)"
7,pivot,unique,None,{PremiumPaid},{},1,0,"(0.99, 0.99)"
8,pivot,std,None,{PremiumPaid},{all},1,1,"(0.99, 0.99)"
9,pivot,count,None,{ClientID},{all},1,1,"(0.99, 0.99)"


### Interpreting Results:

* We can see that the name given is "pivot" - etiq thinks the profitability data is a pivot based on the premiums table.
* We can spot the differences between the two tables:
  * The count is different between these tables (correct)
  * There are no missing `ClientID` values - this is good and an indication our pivot was correct.
  * The count, minimum, maximum, mean and median values are different *but* the sum is correct which suggests our aggregation is correct.

Note too that we've only tested the fields which are common to both tables.

## Other Data Relationships

There are 4 different types of data relationship we detect:

* [pivot](#pivot) - The data is derived from one or more tables. We've seen this above.
* [replica](#replica) - The data is has the same columns but different data - e.g. Sales data from month to month.
* [sampling](#sampling) - The data is a sample of a larger dataset.
* [part](#part) - The data is a part of the given data - it has a subset of columns from another dataset but the same data.

### Replica

Let's explore a replica dataset relationship. We now check that a new claims dataset for different months and amounts is similar.

In [9]:
replica_claims_df = claims_df.copy()

replica_claims_df.Month = np.random.randint(1, 12, size=claims_df.shape[0])
replica_claims_df.Amount = np.random.choice([0.5, 1.0, 2.0], size=claims_df.shape[0])

replica_claims_df

,ClaimID,ClientID,Month,Amount
0,A001,C01,9,0.5
1,A002,C02,8,2.0
2,A003,C01,11,1.0
3,A004,C03,5,1.0
4,A005,C05,1,0.5
5,A006,C05,4,0.5
6,A007,C01,3,2.0
7,A008,C02,8,1.0
8,A009,C02,11,0.5
9,A005,C04,6,0.5


In [10]:
replica_claims_dataset = etiq.SimpleDatasetBuilder.datasets(validation_features=replica_claims_df)
replica_claims_snapshot = project.snapshots.create(name="Replica Claims", dataset=replica_claims_dataset, model=None)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

INFO:etiq.charting:Created histogram summary of data (4 fields)


In [11]:
# Now let's do a fingerprint scan

segments, issues, aggregate_issues = replica_claims_snapshot.scan_fingerprints(
    claims_snapshot, margin_per_field={"Amount": 0.7, "Month": 0.7}
)

display(issues)
display(aggregate_issues)

INFO:etiq.pipeline.BasePipeline0999:Starting pipeline
INFO:etiq.pipeline.BasePipeline0999:Completed pipeline


,name,feature,segment,measure,measure_value,metric,metric_value,threshold,value,record
0,replica,Amount,all,None,NaN,median,0.5,"(0.7, 0.7)",None,None
1,replica,Month,all,None,NaN,median,5.0,"(0.7, 0.7)",None,None


,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,replica,count,None,{Amount},{},1,0,"(0.7, 0.7)"
1,replica,min,None,{Amount},{},1,0,"(0.7, 0.7)"
2,replica,max,None,{Amount},{},1,0,"(0.7, 0.7)"
3,replica,mean,None,{Amount},{},1,0,"(0.7, 0.7)"
4,replica,median,None,{Amount},{all},1,1,"(0.7, 0.7)"
5,replica,missing,None,{Amount},{},1,0,"(0.7, 0.7)"
6,replica,sum,None,{Amount},{},1,0,"(0.7, 0.7)"
7,replica,unique,None,{Amount},{},1,0,"(0.7, 0.7)"
8,replica,std,None,{Amount},{},1,0,"(0.7, 0.7)"
9,replica,count,None,{ClientID},{},1,0,"(0.99, 0.99)"


### Interpretation

We correctly determine this as a replica of the first "claims" dataset. However as the dataset size is so small, some of the metrics we use won't match closely enough.

## Sampling

We can also check that a dataset is just a sample of a larger dataset.

In [12]:
large_df = pd.DataFrame({
    "A": np.random.randint(0, 100, size=10_000),
    "B": np.random.choice(["Mon", "Tue", "Wed"], size=10_000),
    "C": np.random.randint(0, 3, size=10_000),
})

sample_df = large_df.sample(1000)

large_dataset = etiq.SimpleDatasetBuilder.datasets(validation_features=large_df)
sample_dataset = etiq.SimpleDatasetBuilder.datasets(validation_features=sample_df)

large_snapshot = project.snapshots.create(name="Sample Large Dataset", dataset=large_dataset, model=None)
sample_snapshot = project.snapshots.create(name="Sample Dataset", dataset=sample_dataset, model=None)

segments, issues, aggregate_issues = sample_snapshot.scan_fingerprints(large_snapshot, margin=0.9)

display(issues)
display(aggregate_issues)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

""


,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,sample,mean,None,{A},{},1,0,"(0.9, 0.9)"
1,sample,mean,None,{C},{},1,0,"(0.9, 0.9)"


### Interpretation

We can see that it was detected as a sample of the original dataset. And that the mean metric was used as a test for the features.

## Part

We can also detect that a dataset is just a part of an original dataset. Likely just a subset of columns.

In [13]:
part_dataframe = claims_df[["ClaimID", "Month"]]
part_dataset = etiq.SimpleDatasetBuilder.datasets(validation_features=part_dataframe)
part_snapshot = project.snapshots.create(name="Part Snapshot", dataset=part_dataset, model=None)

segments, issues, aggregate_issues = part_snapshot.scan_fingerprints(claims_snapshot)

display(issues)
display(aggregate_issues)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])

INFO:etiq.charting:Histogram summary already created for this data.
INFO:etiq.pipeline.BasePipeline0310:Starting pipeline
INFO:etiq.pipeline.BasePipeline0310:Completed pipeline


""


,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,part,mean,None,{Month},{},1,0,"(0.99, 0.99)"
1,part,sum,None,{Month},{},1,0,"(0.99, 0.99)"


### Interpretation

We see that our dataset is a part of the other dataset and that the mean and sum metrics have been used to assert similarity.

## Margin Change

In all our tests, we've used a single value for specifying the margin - 0.99, the default value. This isn't the best value in each case:

* Our data may vary more than this.
* The "sameness" of our features may vary on a feature to feature basis.

The `scan_fingerprints()` method accepts an argument `margin` and `margin_per_field`. 

Let's see it in action. We'll define a dataset which has two parts - fields we expect to vary in metric and those that don't.

For example with the claims data. If this was data for one month, then for a different month we'd expect:

* Different claim ids
* Different customer ids.
* Similar distribution of months!
* A similar distribution of claim amounts!


In [14]:
size = 10
next_months_claims = pd.DataFrame({
    "ClaimID": [f"A{n:000}" for n in np.random.randint(0, 100, size=size)],
    "ClientID": [f"C{n:00}" for n in np.random.randint(0, 100, size=size)],
    "Month": np.random.randint(1, 12, size=size),
    "Amount": np.random.choice([0.5, 1.0, 2], size=size),
})

next_months_claims

,ClaimID,ClientID,Month,Amount
0,A48,C48,7,2.0
1,A14,C13,1,0.5
2,A55,C55,11,1.0
3,A39,C60,10,1.0
4,A60,C59,8,2.0
5,A36,C76,1,1.0
6,A23,C85,6,1.0
7,A43,C64,11,0.5
8,A4,C86,3,2.0
9,A72,C39,11,1.0


In [15]:
# Naively scanning as before:
next_months_dataset = etiq.SimpleDatasetBuilder.datasets(validation_features=next_months_claims)
next_months_snapshot = project.snapshots.create(name="Next Months Claims", dataset=next_months_dataset, model=None)

segments, issues, aggregate_issues = next_months_snapshot.scan_fingerprints(claims_snapshot)

aggregate_issues[aggregate_issues.issues_found > 0]

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])

INFO:etiq.charting:Created histogram summary of data (4 fields)
INFO:etiq.pipeline.BasePipeline0533:Starting pipeline
INFO:etiq.pipeline.BasePipeline0533:Completed pipeline


,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
3,replica,mean,None,{Amount},{all},1,1,"(0.99, 0.99)"
4,replica,median,None,{Amount},{all},1,1,"(0.99, 0.99)"
6,replica,sum,None,{Amount},{all},1,1,"(0.99, 0.99)"
8,replica,std,None,{Amount},{all},1,1,"(0.99, 0.99)"
11,replica,unique,None,{ClientID},{all},1,1,"(0.99, 0.99)"
14,replica,unique,None,{ClaimID},{all},1,1,"(0.99, 0.99)"
17,replica,max,None,{Month},{all},1,1,"(0.99, 0.99)"
18,replica,mean,None,{Month},{all},1,1,"(0.99, 0.99)"
19,replica,median,None,{Month},{all},1,1,"(0.99, 0.99)"
21,replica,sum,None,{Month},{all},1,1,"(0.99, 0.99)"


So we're getting a lot of noise here because of the above expected differences.

Let's cut that down by applying a margin to our fingerprint.

All fields will receive a margin of 0.1 but we are still interested in the "Amount" field which will keep a high margin:

In [16]:
segments, issues, aggregate_issues = next_months_snapshot.scan_fingerprints(
    claims_snapshot,
    margin_per_field={
        "Amount": 0.90,
    },
    margin=0.1,
)

aggregate_issues[aggregate_issues.issues_found > 0]

INFO:etiq.pipeline.BasePipeline0203:Starting pipeline
INFO:etiq.pipeline.BasePipeline0203:Completed pipeline


,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
3,replica,mean,None,{Amount},{all},1,1,"(0.9, 0.9)"
4,replica,median,None,{Amount},{all},1,1,"(0.9, 0.9)"
6,replica,sum,None,{Amount},{all},1,1,"(0.9, 0.9)"
8,replica,std,None,{Amount},{all},1,1,"(0.9, 0.9)"
11,replica,unique,None,{ClientID},{all},1,1,"(0.1, 0.1)"


We still have issues with the "Amount" field due to the very small amount of data we're using for this demo. But it serves as an example of turning down "noise" on fields which we'd expect the change.

## API Reference

As seen above, basic usage is simple, for a given snapshot, we just call `scan_fingerprints()` with the other snapshot representing another dataset to see how they compare.

```python
new_dataset_snapshot.scan_fingerprints(original_dataset_snapshot)
```

The method has a few optional arguments:

* `margin` - A float which we use to indicate the margin by which we allow fields to match. For example, a value of 0.99 indicates we expect the metrics on both tables to come within 99% of each other.
  The default is 0.99.

```python
    new_dataset_snapshot.scan_fingerprints(original_dataset_snapshot, margin=0.9)
```

* `margin_per_field` - Some fields may vary more than others so a single margin doesn't make sense. This argument is a dictionary whose keys are string column names and values are the margin for that field.

```python
    # We decide that "PremiumPaid" should match exactly. The other features will get the usual 0.9.
    new_dataset_snapshot.scan_fingerprints(original_dataset_snapshot, margin_per_field={"PremiumPaid": 0})
```

* `metrics` - You may want to limit the metrics to a subset of the above named metrics as given in **Table 1**.

```python
    new_dataset_snapshot.scan_fingerprints(original_dataset_snapshot, metrics=["sum", "avg"])
```
